In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Creating and running a tensorflow session

In [2]:
#Assigning a variable in the computational graph and then running the session
x = tf.Variable(3,name='x')
y = tf.Variable(4,name='y')
f = x*x*y + y + 2
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
sess.run(f)

42

In [3]:
#Assignning using constant doesnot require placing the values in the computation graph
x = tf.constant(3)
y = tf.constant(4)
f = x*x*y + y + 2
sess = tf.Session()
sess.run(f)

42

In [4]:
#using global variables_initializer() and InteractiveSession()
x = tf.Variable(3,name='x')
y = tf.Variable(4,name='y')
f = x*x*y + y + 2
init = tf.global_variables_initializer() #prepare an init node
sess = tf.InteractiveSession()
sess.run(init)
sess.run(f)

42

In [5]:
#using global variables_initializer() and Session()
x = tf.Variable(3,name='x')
y = tf.Variable(4,name='y')
f = x*x*y + y + 2
init = tf.global_variables_initializer() #creating a node in the graph
sess = tf.Session()
sess.run(init)#initialising all the variables
sess.run(f)

42

### Managing Graphs

In [6]:
#Anything that we create is automatically added to the default graph
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [7]:
#creating independent graph
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph() )

True
False


### Life cycle of a Node value

In [8]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x + 3
with tf.Session() as sess:
    print(sess.run(y))
    print(sess.run(z))

10
8


### Linear Regression with Tensorflow

In [9]:
housing = pd.read_csv('regression.csv')
housing = housing.drop(['Unnamed: 0'],axis=1)
housing.head()

,0,1,2,3,4,5,6,7
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [10]:
housing_labels = pd.read_csv('targets.csv')
housing_labels = housing_labels.drop('Unnamed: 0',axis=1)

In [11]:
housing_labels = np.array(housing_labels)

In [12]:
housing = np.array(housing)

In [13]:
m,n = housing.shape

In [14]:
housing_adding_bias = np.c_[np.ones((m,1)),housing]

In [15]:
X = tf.constant(housing_adding_bias,dtype=tf.float64,name="X")
y = tf.constant(housing_labels,dtype=tf.float64,name="y")

In [16]:
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

In [17]:
sess = tf.Session()
sess.run(theta)

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654265e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

In [18]:
#Comparing with numpy
X = housing_adding_bias
y = housing_labels
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
theta_numpy

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654265e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

### Implementing Gradient descent

#### 1.Manually computing the gradients

In [19]:
 from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
housing_std = scaler.fit_transform(housing)
std_housing_add_bias = np.c_[np.ones((m,1)),housing_std]

In [20]:
#implementing batch gradient descent
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(std_housing_add_bias,dtype=tf.float32,name="X")
y = tf.constant(housing_labels,dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name='theta')#tf.random_uniform works same like np.random.rand()
y_pred = tf.matmul(X,theta,name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')#tf.reduce_mean finds the mean
gradients = 2/m * tf.matmul(tf.transpose(X),error)
training_op = tf.assign(theta,theta - learning_rate * gradients)#tf.assign()creates a node that will assign a new value to theta

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", sess.run(mse))
        sess.run(training_op)
    
best_theta = sess.run(theta)

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.53737885
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293705


In [21]:
best_theta

array([[ 2.06855226e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44081801e-04],
       [-3.91945131e-02],
       [-8.61356556e-01],
       [-8.23479712e-01]], dtype=float32)

#### 2.Using autodiff

In [22]:
#implementing batch gradient descent
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(std_housing_add_bias,dtype=tf.float32,name="X")
y = tf.constant(housing_labels,dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name='theta')#tf.random_uniform works same like np.random.rand()
y_pred = tf.matmul(X,theta,name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')#tf.reduce_mean finds the mean
gradients = tf.gradients(mse,[theta])[0]# instead of 2/m * tf.matmul(tf.transpose(X),error)
training_op = tf.assign(theta,theta - learning_rate * gradients)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", sess.run(mse))
        sess.run(training_op)
    
best_theta = sess.run(theta)

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373789
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293704


In [23]:
best_theta

array([[ 2.06855249e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44078017e-04],
       [-3.91945094e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

#### 3.Using an Optimizer

In [24]:
#implementing batch gradient descent
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(std_housing_add_bias,dtype=tf.float32,name="X")
y = tf.constant(housing_labels,dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name='theta')#tf.random_uniform works same like np.random.rand()
y_pred = tf.matmul(X,theta,name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')#tf.reduce_mean finds the mean
#gradients = tf.gradients(mse,[theta])[0]# instead of 2/m * tf.matmul(tf.transpose(X),error)
#training_op = tf.assign(theta,theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", sess.run(mse))
        sess.run(training_op)
    
best_theta = sess.run(theta)

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373789
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293704


### Using placeholders to train mini-batches

In [25]:
#creating placeholders
A = tf.placeholder(tf.float32,shape=(None,3))
B = A + 5
print(sess.run(B,feed_dict={A:[[1,2,3]]}))
print(sess.run(B,feed_dict={A:[[4,5,6],[7,8,9]]}))

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [26]:
#implementing mini -batch gradient descent
n_epochs = 10
learning_rate = 0.01
batch_size=100
n_batches = int(np.ceil(m/batch_size))

X = tf.placeholder(dtype=tf.float32,shape=(None,n+1),name="X")
y = tf.placeholder(dtype=tf.float32,shape=(None,1),name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name='theta')#tf.random_uniform works same like np.random.rand()
y_pred = tf.matmul(X,theta,name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')#tf.reduce_mean finds the mean
#gradients = tf.gradients(mse,[theta])[0]# instead of 2/m * tf.matmul(tf.transpose(X),error)
#training_op = tf.assign(theta,theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
#creating a function to get the minibatches
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = std_housing_add_bias[indices] # not shown
    y_batch = housing_labels[indices] # not shown
    return X_batch, y_batch

for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

best_theta = sess.run(theta)

In [27]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.1173173 ],
       [-0.22739051],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)

### Saving and Restoring Models

In [28]:
#implementing batch gradient descent
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(std_housing_add_bias,dtype=tf.float32,name="X")
y = tf.constant(housing_labels,dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name='theta')#tf.random_uniform works same like np.random.rand()
y_pred = tf.matmul(X,theta,name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')#tf.reduce_mean finds the mean
#gradients = tf.gradients(mse,[theta])[0]# instead of 2/m * tf.matmul(tf.transpose(X),error)
#training_op = tf.assign(theta,theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
saver = tf.train.Saver()
for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", sess.run(mse))
            save_path = saver.save(sess, "C:/Users/HP/Hands on Machine learning with sckitlearn and tensor flow/End to end ML project/gradientdescent.ckpt")
        sess.run(training_op)
    
best_theta = sess.run(theta)
save_path = saver.save(sess, "C:/Users/HP/Hands on Machine learning with sckitlearn and tensor flow/End to end ML project/gradientdescent.ckpt")

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373789
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293704


In [29]:
best_theta

array([[ 2.06855249e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44078017e-04],
       [-3.91945094e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

In [30]:
sess = tf.Session()
saver.restore(sess, "C:/Users/HP/Hands on Machine learning with sckitlearn and tensor flow/End to end ML project/gradientdescent.ckpt")
best_theta_restored = sess.run(theta)

INFO:tensorflow:Restoring parameters from C:/Users/HP/Hands on Machine learning with sckitlearn and tensor flow/End to end ML project/gradientdescent.ckpt


In [31]:
best_theta_restored

array([[ 2.06855249e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44078017e-04],
       [-3.91945094e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

In [32]:
np.allclose(best_theta, best_theta_restored)

True

In [33]:
#saving and restoring a particular variable
#implementing batch gradient descent
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(std_housing_add_bias,dtype=tf.float32,name="X")
y = tf.constant(housing_labels,dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name='theta')#tf.random_uniform works same like np.random.rand()
y_pred = tf.matmul(X,theta,name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')#tf.reduce_mean finds the mean
#gradients = tf.gradients(mse,[theta])[0]# instead of 2/m * tf.matmul(tf.transpose(X),error)
#training_op = tf.assign(theta,theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
saver = tf.train.Saver({'weights':theta})
for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", sess.run(mse))
            save_path = saver.save(sess, "C:/Users/HP/Hands on Machine learning with sckitlearn and tensor flow/End to end ML project/gradientdescent.ckpt")
        sess.run(training_op)
    
best_theta = sess.run(theta)
save_path = saver.save(sess, "C:/Users/HP/Hands on Machine learning with sckitlearn and tensor flow/End to end ML project/gradientdescent.ckpt")

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373789
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293704


In [34]:
best_theta

array([[ 2.06855249e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44078017e-04],
       [-3.91945094e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

In [35]:
sess = tf.Session()
saver.restore(sess, "C:/Users/HP/Hands on Machine learning with sckitlearn and tensor flow/End to end ML project/gradientdescent.ckpt")
best_theta_restored = sess.run(theta)

INFO:tensorflow:Restoring parameters from C:/Users/HP/Hands on Machine learning with sckitlearn and tensor flow/End to end ML project/gradientdescent.ckpt


In [36]:
best_theta_restored

array([[ 2.06855249e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44078017e-04],
       [-3.91945094e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

In [37]:
np.allclose(best_theta, best_theta_restored)

True

## Logistic Regression using tensorflow

In [38]:
iris = pd.read_csv("iris.csv")
dat = iris[['SepalLength','SepalWidth','Name']]
dat = dat.loc[dat['Name']!='Iris-virginica',:]

In [39]:
from sklearn.preprocessing import StandardScaler

In [40]:
scaler = StandardScaler()
x = scaler.fit_transform(dat.iloc[:,:-1])
Y = np.where(dat['Name']=='Iris-setosa',1,0)

In [41]:
x.shape[0]

100

In [42]:
class LogisticRegression:
    def __init__(self,n_epochs=1000,n_inputs=2,learning_rate=0.01):
        self.epochs = n_epochs
        self.lr = learning_rate
        self.W = tf.Variable(np.random.randn(n_inputs,1), dtype=tf.float32,name = 'W')#shape = (n_inputs,1)
        self.b = tf.Variable(tf.zeros(1), dtype=tf.float32,name='b')
    def fit(self,x,Y):
        X = tf.constant(x.T,dtype=tf.float32,name="X")
        y = tf.constant(Y.reshape(1,x.shape[0]),dtype=tf.float32,name="y")
        Z = tf.matmul(tf.transpose(self.W),X) + self.b
        A = tf.sigmoid(Z)
        #cost = tf.reduce_mean(-(y*tf.log(A)+(1-y)*tf.log(1-A)))
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Z,  labels = y))
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = self.lr)
        training_op = optimizer.minimize(cost)
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        for epoch in range(self.epochs):
            if epoch % 100 == 0:
                print("Epoch", epoch, "cost =", sess.run(cost))
            sess.run(training_op)
        self.W = sess.run(self.W)
        self.b = sess.run(self.b)
    def predict(self,x,threshold=0.5):
        X_test = tf.constant(x.T,dtype=tf.float32,name="X_test")
        Z = tf.matmul(tf.transpose(self.W),X_test) + self.b
        A = tf.sigmoid(Z)
        sess = tf.Session()
        prob = sess.run(A)
        return prob>threshold
    def predict_proba(self,x):
        X_test = tf.constant(x.T,dtype=tf.float32,name="X_test")
        Z = tf.matmul(tf.transpose(self.W),X_test) + self.b
        A = tf.sigmoid(Z)
        sess = tf.Session()
        prob = sess.run(A)
        return prob
    def scores(self,x):
        X_test = tf.constant(x.T,dtype=tf.float32,name="X_test")
        Z = tf.matmul(tf.transpose(self.W),X_test) + self.b
        A = tf.sigmoid(Z)
        sess = tf.Session()
        best_scores = sess.run(Z)
        return best_scores

In [43]:
logit = LogisticRegression()

In [44]:
logit.fit(x,Y)

Epoch 0 cost = 0.7122419
Epoch 100 cost = 0.51594085
Epoch 200 cost = 0.40080333
Epoch 300 cost = 0.32888138
Epoch 400 cost = 0.2806491
Epoch 500 cost = 0.24628246
Epoch 600 cost = 0.22058849
Epoch 700 cost = 0.20063834
Epoch 800 cost = 0.18467644
Epoch 900 cost = 0.17159346


In [45]:
logit.predict(x)

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False]])

In [46]:
logit.scores(x)

array([[ 2.0097692 ,  1.160053  ,  2.2222908 ,  2.2071369 ,  2.540888  ,
         2.3283665 ,  3.0265458 ,  1.9946156 ,  2.1768298 ,  1.4331893 ,
         1.7820939 ,  2.5105805 ,  1.4180355 ,  2.7079484 ,  1.5695722 ,
         2.9200997 ,  2.3283665 ,  2.0097692 ,  1.2812824 ,  2.829178  ,
         0.96268505,  2.5560417 ,  3.5728183 ,  1.4634966 ,  2.5105805 ,
         0.90207046,  1.9946156 ,  1.7517865 ,  1.4786502 ,  2.2222908 ,
         1.6911718 ,  0.96268505,  3.3906043 ,  2.8897927 ,  1.4331893 ,
         1.4483429 ,  0.9778388 ,  1.4331893 ,  2.449966  ,  1.7366328 ,
         2.2677517 ,  0.28002936,  2.9962387 ,  2.2677517 ,  2.829178  ,
         1.4180355 ,  2.829178  ,  2.4802732 ,  2.0400765 ,  1.7214792 ,
        -3.711309  , -2.1634135 , -3.7264628 , -2.2997968 , -3.5139413 ,
        -1.4500805 , -1.6322947 , -0.4787646 , -3.4987876 , -0.4333037 ,
        -1.8292925 , -1.4197731 , -3.8628461 , -2.2088747 , -0.9189616 ,
        -3.2104979 , -0.6458253 , -1.9811994 , -4.3

In [47]:
logit.predict_proba(x)

array([[0.88181895, 0.76134235, 0.9022334 , 0.9008886 , 0.926959  ,
        0.9111993 , 0.95375913, 0.8802306 , 0.89814943, 0.8073977 ,
        0.85595524, 0.9248802 , 0.8050303 , 0.9374941 , 0.8277226 ,
        0.9488312 , 0.9111993 , 0.88181895, 0.782668  , 0.9442323 ,
        0.72365904, 0.9279784 , 0.9726901 , 0.8120669 , 0.9248802 ,
        0.71137476, 0.8802306 , 0.852178  , 0.8143686 , 0.9022334 ,
        0.84437823, 0.72365904, 0.9674096 , 0.94733953, 0.8073977 ,
        0.8097432 , 0.7266792 , 0.8073977 , 0.9205589 , 0.8502589 ,
        0.9061708 , 0.5695534 , 0.9524039 , 0.9061708 , 0.9442323 ,
        0.8050303 , 0.9442323 , 0.9227473 , 0.8849411 , 0.84831923,
        0.02386218, 0.10308442, 0.02351174, 0.09113979, 0.02891815,
        0.18998918, 0.16351627, 0.38254392, 0.02934675, 0.39333773,
        0.13832258, 0.19469716, 0.02057586, 0.09895637, 0.28516954,
        0.03877258, 0.3439309 , 0.12119104, 0.01238494, 0.11800007,
        0.29452646, 0.07712895, 0.02151227, 0.07

In [48]:
def input_hidden(x,Y,hidden_neurons,n_inputs):
    W1 = tf.Variable(np.random.randn(hidden_neurons,n_inputs), dtype=tf.float32,name = 'W1')#shape = (n_inputs,1)
    b1 = tf.Variable(tf.zeros([hidden_neurons,1]), dtype=tf.float32,name='b1')
    X = tf.constant(x.T,dtype=tf.float32,name="X")
    y = tf.constant(Y.reshape(1,x.shape[0]),dtype=tf.float32,name="y")
    Z = tf.matmul(W1,X) + b1
    A = tf.nn.relu(Z)
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    return sess.run(W1),sess.run(b1),sess.run(A)

In [49]:
weights,bias,activations = input_hidden(x,Y,hidden_neurons=2,n_inputs=2)

In [50]:
def hidden_output(hidden_activations,Y,n_outputs,hidden_neurons):
    W2 = tf.Variable(np.random.randn(n_outputs,hidden_neurons), dtype=tf.float32,name = 'W2')#shape = (n_inputs,1)
    b2 = tf.Variable(tf.zeros([n_outputs,1]), dtype=tf.float32,name='b2')
    hidden_activations = tf.constant(hidden_activations,dtype=tf.float32,name="hidden_activations")
    y = tf.constant(Y.reshape(1,x.shape[0]),dtype=tf.float32,name="y")
    Z = tf.matmul(W2,hidden_activations) + b2
    A = tf.sigmoid(Z)
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    return sess.run(W2),sess.run(b2),sess.run(Z)

In [51]:
weights,bias,logits = hidden_output(activations,Y,n_outputs=1,hidden_neurons=2)

In [52]:
logits.shape

(1, 100)

In [53]:
def cost(Y,z):
    y = tf.constant(Y.reshape(1,x.shape[0]),dtype=tf.float32,name="y")
    Z = tf.constant(z.reshape(1,x.shape[0]),dtype=tf.float32,name="Z")
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Z,  labels = y))
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    return sess.run(cost)

In [54]:
cost(Y,logits)

1.0209799

In [55]:
class MLP:
    def __init__(self,n_epochs=1000,n_inputs=2,hidden_neurons=2,n_outputs=1,learning_rate=0.01):
        self.epochs = n_epochs
        self.lr = learning_rate
        self.W1 = tf.Variable(np.random.randn(hidden_neurons,n_inputs), dtype=tf.float32,name = 'W1')#shape = (n_inputs,1)
        self.b1 = tf.Variable(tf.zeros([hidden_neurons,1]), dtype=tf.float32,name='b1')
        self.W2 = tf.Variable(np.random.randn(n_outputs,hidden_neurons), dtype=tf.float32,name = 'W2')#shape = (n_inputs,1)
        self.b2 = tf.Variable(tf.zeros([n_outputs,1]), dtype=tf.float32,name='b2')
    def fit(self,x,Y):
        X = tf.constant(x.T,dtype=tf.float32,name="X")
        y = tf.constant(Y.reshape(1,x.shape[0]),dtype=tf.float32,name="y")
        Z1 = tf.add(tf.matmul(self.W1,X),self.b1)
        A1 = tf.nn.relu(Z1)
        Z2 = tf.add(tf.matmul(self.W2,A1),self.b2)
        #Z = tf.constant(Z2,dtype=tf.float32,name="Z")
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Z2,  labels = y))
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = self.lr)
        training_op = optimizer.minimize(cost)
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        for epoch in range(self.epochs):
            if epoch % 100 == 0:
                print("Epoch", epoch, "cost =", sess.run(cost))
            sess.run(training_op)
        self.W1 = sess.run(self.W1)
        self.b1 = sess.run(self.b1)
        self.W2 = sess.run(self.W2)
        self.b2 = sess.run(self.b2)
    def predict(self,x,threshold=0.5):
        X_test = tf.constant(x.T,dtype=tf.float32,name="X_test")
        Z1 = tf.add(tf.matmul(self.W1,X_test),self.b1)
        A1 = tf.nn.relu(Z1)
        Z2 = tf.add(tf.matmul(self.W2,A1),self.b2)
        A2 = tf.sigmoid(Z2)
        sess = tf.Session()
        prob = sess.run(A2)
        return prob>threshold
    def predict_proba(self,x):
        X_test = tf.constant(x.T,dtype=tf.float32,name="X_test")
        Z1 = tf.add(tf.matmul(self.W1,X_test),self.b1)
        A1 = tf.nn.relu(Z1)
        Z2 = tf.add(tf.matmul(self.W2,A1),self.b2)
        A2 = tf.sigmoid(Z2)
        sess = tf.Session()
        prob = sess.run(A2)
        return prob

In [56]:
mlp = MLP()

In [57]:
mlp.fit(x,Y)

Epoch 0 cost = 0.87532973
Epoch 100 cost = 0.6549813
Epoch 200 cost = 0.52948296
Epoch 300 cost = 0.4295184
Epoch 400 cost = 0.3434387
Epoch 500 cost = 0.27722684
Epoch 600 cost = 0.22775063
Epoch 700 cost = 0.18921483
Epoch 800 cost = 0.15910025
Epoch 900 cost = 0.1346353


In [58]:
mlp.predict(x,0.5)

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False]])

In [59]:
mlp.predict_proba(x)

array([[0.9536948 , 0.87792635, 0.9696456 , 0.97019684, 0.9774809 ,
        0.96550095, 0.98977685, 0.95452213, 0.9712704 , 0.91183525,
        0.9311881 , 0.9782979 , 0.9133426 , 0.98615927, 0.89717156,
        0.98204297, 0.96550095, 0.9536948 , 0.8607781 , 0.9839368 ,
        0.8198221 , 0.9770611 , 0.9950305 , 0.90874946, 0.9782979 ,
        0.83071834, 0.95452213, 0.9335707 , 0.9071703 , 0.9696456 ,
        0.9381073 , 0.8198221 , 0.9920197 , 0.9826974 , 0.91183525,
        0.9103043 , 0.81701386, 0.91183525, 0.97984594, 0.934733  ,
        0.9679319 , 0.7228374 , 0.9901523 , 0.9679319 , 0.9839368 ,
        0.9133426 , 0.9839368 , 0.979086  , 0.95199686, 0.93587637,
        0.03433632, 0.0858851 , 0.03644367, 0.14073408, 0.05080245,
        0.16354051, 0.1088244 , 0.4484467 , 0.0479061 , 0.40641016,
        0.21415305, 0.14734039, 0.06185429, 0.10158838, 0.2026235 ,
        0.04969048, 0.23777513, 0.13075982, 0.04552011, 0.14544788,
        0.17433305, 0.09280181, 0.05193796, 0.09

In [60]:
final =  pd.read_csv("final.csv",sep='\t')
dat = final.drop('Unnamed: 0',axis=1)

In [61]:
dat.head()

,SEX,EDUCATION,MARRIAGE,unknown -2,duly paid,unknown 0,delay by one month,delay by two month,delay by three month,delay by four months and above,LIMIT_BAL,Previous_pmt_Sum,AGE,Mean_bill_Amount,default payment next month
0,2,2,1,1,1,0,0,1,0,0,20000,689,24,1284.000000,1
1,2,2,2,0,1,1,0,1,0,0,120000,5000,26,2846.166667,1
2,2,2,2,0,0,1,0,0,0,0,90000,11018,34,16942.166667,0
3,2,2,1,0,0,1,0,0,0,0,50000,8388,37,38555.666667,0
4,1,2,1,0,1,1,0,0,0,0,50000,59049,57,18223.166667,0


In [62]:
dat = dat.copy()
dat = dat.loc[dat['Previous_pmt_Sum']<=73739.125]
dat = dat.loc[dat['LIMIT_BAL']<=525000]

In [63]:
#writing a function to randomly sample equal no. of samples of defaulter and non defaulter for the training set.
#The remaining samples can be kept for the trainig set.
#0 - majority , 1 - minority
def train_test_split(data,target_column,majorityclass_samples,minorityclass_samples):
    x = majorityclass_samples#no of samples for majority class
    y = minorityclass_samples#no of samples for minority class
    dat_0 = dat.loc[dat[target_column]==0,:]
    dat_1 = dat.loc[dat[target_column]==1,:]
    dat_0.index = np.arange(len(dat_0))
    dat_1.index = np.arange(len(dat_1))
    nodefault_index_train = np.random.permutation(len(dat_0))[:x]
    nodefault_index_test = np.random.permutation(len(dat_0))[x:]
    default_index_train = np.random.permutation(len(dat_1))[:y]
    default_index_test = np.random.permutation(len(dat_1))[y:]
    nodefault_train = dat_0.loc[nodefault_index_train]
    nodefault_test = dat_0.loc[nodefault_index_test]
    default_train = dat_1.loc[default_index_train]
    default_test = dat_1.loc[default_index_test]
    train = pd.concat([nodefault_train,default_train])
    test = pd.concat([nodefault_test,default_test])
    train.index = np.arange(len(train))
    test.index = np.arange(len(test))
    return train,test

In [64]:
#convert categorical columns to one hot array
def convert_to_onehotarray(dat,target_columns):
    encoder = LabelBinarizer()
    dat = dat[target_columns]
    d={}
    l=[]
    d_attr = {}
    for i in target_columns:
        d[i] = encoder.fit_transform(dat[i])
        d_attr[i] = list(encoder.classes_)
        l.append(d[i])
    return np.concatenate(l,axis=1),d_attr 

In [65]:
#combining categorical and continous columns to form a array ready for training.
def combine_cat_num(dat,cat_attributes,num_attributes):
    cat,encoder_attributes = convert_to_onehotarray(dat,cat_attributes)
    d = {}
    l = []
    for i in num_attributes:
        d[i] = dat[i].values.reshape(len(dat[i]),1)
        l.append(d[i])
    num = np.concatenate(l,axis=1)
    final = np.concatenate([cat,num],axis=1)
    return final,encoder_attributes

In [66]:
cat_attributes = ['SEX', 'EDUCATION', 'MARRIAGE', 'unknown -2', 'duly paid', 'unknown 0','delay by one month', 'delay by two month', 'delay by three month','delay by four months and above']
num_attributes = ['LIMIT_BAL', 'Previous_pmt_Sum','AGE','Mean_bill_Amount']

In [67]:
train,test = train_test_split(dat,'default payment next month',5000,5000)

In [68]:
#reshuffling the training and testing set 
train = train.loc[np.random.permutation(len(train))]
test = test.loc[np.random.permutation(len(test))]

In [69]:
from sklearn.preprocessing import LabelBinarizer

In [70]:
train_x,train_cat_attr = combine_cat_num(train,cat_attributes,num_attributes)
train_y = train.iloc[:,-1]
test_x,test_cat_attr = combine_cat_num(test,cat_attributes,num_attributes)
test_y = test.iloc[:,-1]

In [71]:
def transform(X):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    return np.c_[train_x[:,:15],scaler.fit_transform(train_x[:,-4:])]   

In [72]:
x = transform(train_x)
Y = np.array(train_y)

In [73]:
x.shape

(10000, 19)

In [74]:
mlp = MLP(n_epochs=10000,n_inputs=19,hidden_neurons=15,n_outputs=1,learning_rate=0.05)

In [75]:
mlp.fit(x,Y)

Epoch 0 cost = 2.500333
Epoch 100 cost = 0.70150137
Epoch 200 cost = 0.65714127
Epoch 300 cost = 0.64078236
Epoch 400 cost = 0.6324878
Epoch 500 cost = 0.6272386
Epoch 600 cost = 0.62328327
Epoch 700 cost = 0.6200538
Epoch 800 cost = 0.61731535
Epoch 900 cost = 0.6148458
Epoch 1000 cost = 0.6126961
Epoch 1100 cost = 0.6107268
Epoch 1200 cost = 0.60896665
Epoch 1300 cost = 0.6073298
Epoch 1400 cost = 0.6057739
Epoch 1500 cost = 0.60435957
Epoch 1600 cost = 0.60304445
Epoch 1700 cost = 0.6018725
Epoch 1800 cost = 0.6008398
Epoch 1900 cost = 0.5999019
Epoch 2000 cost = 0.5990423
Epoch 2100 cost = 0.5982249
Epoch 2200 cost = 0.5974679
Epoch 2300 cost = 0.59677196
Epoch 2400 cost = 0.5961537
Epoch 2500 cost = 0.59559715
Epoch 2600 cost = 0.59505826
Epoch 2700 cost = 0.5945503
Epoch 2800 cost = 0.5940752
Epoch 2900 cost = 0.5936162
Epoch 3000 cost = 0.5931951
Epoch 3100 cost = 0.59278595
Epoch 3200 cost = 0.59236616
Epoch 3300 cost = 0.5919771
Epoch 3400 cost = 0.5916274
Epoch 3500 cost = 0.

In [79]:
y_predicted = np.where(mlp.predict(x,0.5)==False,0,1)

In [80]:
y_actual = np.array(Y)

In [81]:
d = pd.DataFrame(np.c_[y_actual.reshape(10000,1),y_predicted.reshape(10000,1)])
d.columns = ['Actual','Predicted']
d.head()

,Actual,Predicted
0,1,1
1,1,0
2,1,1
3,0,0
4,1,1


In [82]:
d['Actual'].value_counts()

1    5000
0    5000
Name: Actual, dtype: int64

In [83]:
d['Predicted'].value_counts()

0    5386
1    4614
Name: Predicted, dtype: int64

In [86]:
def confusion_matrix(train_y,predicted_train_y):
    from sklearn.metrics import confusion_matrix
    a = pd.DataFrame(confusion_matrix(train_y,predicted_train_y))
    a.columns = ['predicted_0','predicted_1']
    a.index = ['Actual_0','Actual_1']
    return a

In [87]:
confusion_matrix(d['Actual'].values,d['Predicted'].values)

,predicted_0,predicted_1
Actual_0,3706,1294
Actual_1,1680,3320


In [88]:
def precision_recall_f1(actuall_class,predicted_class):
    from sklearn.metrics import precision_score,recall_score,f1_score
    prec = precision_score(actuall_class,predicted_class)
    rec =  recall_score(actuall_class,predicted_class)
    f1 = f1_score(actuall_class,predicted_class)
    return print("precsion-" + str(prec)," Recall-" + str(rec)," F1 score-" + str(f1))

In [89]:
precision_recall_f1(d['Actual'].values,d['Predicted'].values)

precsion-0.7195491980927612  Recall-0.664  F1 score-0.6906594549615145
